In [ ]:
#АЛГОРИТМ РАБОТАЕТ ТОЛЬКО ПОШАГОВО
#pip install haversine
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import ipywidgets as widgets
from IPython import get_ipython
from IPython.display import Image
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles
from ipywidgets import Layout
import requests
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
pd.options.mode.chained_assignment = None #убирает предупреждения

In [ ]:
center = (47.03823025939306, 28.852018195122298)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=7)
marker = Marker(location=center, draggable=True)
m.add_layer(marker)
display(m)
def handle_click(**kwargs):
    print('Широта, долгота: ', kwargs['coordinates'])
marker.on_click(handle_click)
#ЧТОБЫ УЗНАТЬ КООРДИНАТЫ, ПЕРЕМЕСТИТЕ МАРКЕР НА НУЖНУЮ ТОЧКУ И НАЖМИТЕ НА НЕГО

In [ ]:
#Задай координаты точки в десятичном виде (lat, lon) 
X1 = float(input('Задай широту в десятичном виде:\n'))
X2 = float(input('Задай долготу в десятичном виде:\n'))
X = (X1, X2) #(47.03823025939306, 28.852018195122298)

In [ ]:
#ТАБЛИЦА МИЛЕВСКОГО
PATH = "/Users/Legion/"
Image(filename = PATH + "Milevski_Table.jpg", width=827, height=668)  

In [ ]:
#Определение классов открытости по Милевскому для площадки ВЭС
MX = [0] * 8
C_D = ['Север', 'Северо-Восток', 
       'Восток', 'Юго-Восток', 'Юг', 
       'Юго-Запад', 'Запад', 'Северо-Запад']
for i in range(len(C_D)):
    print('Задай коэффициент по таблице Милевского для направления:', C_D[i])
    i = int(i)
    i = i - 1
    MX[i] = int(input())

In [ ]:
Cons = float(input('Задай уровень среднегодового потребления электроэнергии, кВт*ч \n'))

In [ ]:
#Импортирует список ВЭУ
Wind_Turbines = pd.read_excel('Wind_Turbines.xlsx')
WT_Models = list(Wind_Turbines.columns)
WT_Models_Power = list(Wind_Turbines.iloc[52])
WT_Models_Power.remove('Power, kW')
del WT_Models[0]
Select_WT = ()

In [ ]:
def Wind_calc(Select_WT, X, MX):
    #Импортируй список Метеостанций
    Meteostations = pd.read_csv('Meteostations.txt', sep=",", header=None)
    Meteostations.columns = ["Meteostation", "latitude", "longitude"]
    
    #Рассчитай расстояние от точки Х до каждой из Метеостанций
    Rez=()
    lis=()
    i=0
    while i<=len(Meteostations.index)-1:
        Meteo = (Meteostations.iloc[i]['latitude'], 
                 Meteostations.iloc[i]['longitude']) #получить вид (lat, lon)
        Rez = round(haversine(X, Meteo),3)
        lis = np.append([lis],[Rez])
        i = i+1
    
    #Найди индекс минимального расстояния. Выбираем ближайшую метеостанцию
    numb = np.where(lis == min(lis))
    #Результат в виде кортежа(!)
    
    #Прочти данные выбранной метеостанции
    Meteostation_X = pd.read_excel((Meteostations.iloc[int(numb[0])]
                                    ['Meteostation'] + '.xlsx'))
    
    #Прочти данные по классу открытости из БД Флюгер
    Fluger = pd.read_excel('Fluger.xlsx')
    
    #Берем данные для выбранной метеостанции
    Fluger_X = Fluger.loc[numb].drop(['Meteostation'], axis=1)
    
    #Коэффициент k0 для пересчета скорости ветра к условиям площадки ВЭС
    k0=()
    if int(Fluger_X.loc[numb, 'N']) != 1:
        k0 = round((MX / Fluger_X), 3)
    else:
        k0 = Fluger_X
    
    #Пересчет скорости ветра расчетного года к условиям площадки ВЭС
    j=0
    V_VES=()
    while j<len(Meteostation_X):
        loc1 = Meteostation_X.loc[j]['DD']
        loc2 = Meteostation_X.loc[j]['ff']
        if (loc1 >= 0.0 and loc1 < 22.5) or (loc1 >= 337.5 and loc1 <= 360):
            V_VES = np.append([V_VES],[(loc2*k0['N'])])
        elif (loc1 >= 22.5 and loc1 < 67.5):
            V_VES = np.append([V_VES],[(loc2*k0['NE'])])
        elif (loc1 >= 67.5 and loc1 < 112.5):
            V_VES = np.append([V_VES],[(loc2*k0['E'])])
        elif (loc1 >= 112.5 and loc1 < 157.5):
            V_VES = np.append([V_VES],[(loc2*k0['SE'])])
        elif (loc1 >= 157.5 and loc1 < 202.5):
            V_VES = np.append([V_VES],[(loc2*k0['S'])])
        elif (loc1 >= 202.5 and loc1 < 247.5):
            V_VES = np.append([V_VES],[(loc2*k0['SW'])])
        elif (loc1 >= 247.5 and loc1 < 292.5):
            V_VES = np.append([V_VES],[(loc2*k0['W'])])
        elif (loc1 >= 292.5 and loc1 < 337.5):
            V_VES = np.append([V_VES],[(loc2*k0['NW'])])
        elif (loc1 < 0):
            V_VES = np.append([V_VES],[0])
        j = j+1
    VV_VES = pd.DataFrame(data=V_VES, columns=['V_VES'])
    
    #Все в одну БД для расчета
    Meteostation_XX = Meteostation_X.join(VV_VES)
    
    #Рассчитай среднемесячные скорости ветра за расчетный год
    def V_Month_Calc(ind):
        V_Month = sum(Meteostation_XX.loc[Meteostation_XX['Month'] == ind]
                      ['V_VES'])/len(Meteostation_XX.loc
                                     [Meteostation_XX['Month'] == ind]['V_VES'])
        return V_Month
    VV_Month = ()
    for i in range(1, 13):
        VV_Month = np.append([VV_Month], [V_Month_Calc(i)])
    VV_Month = round(pd.DataFrame(data=VV_Month, columns=['V_Month']), 3)
    
    #Коэффиценты для расчета показателя степени m(V) для каждой Метеостанции
    Koef_m = pd.read_excel('Stepeni_m.xlsx')
    Koef_m
    
    #Рассчитай коэффициенты m(V)
    m_VES = ()
    a = Koef_m.loc[numb].drop(['Meteostation'], axis=1)['a']
    b = Koef_m.loc[numb].drop(['Meteostation'], axis=1)['b']
    for i in range (0, 12):
        m_VES = np.append([m_VES], [a * float(VV_Month.iloc[i]) ** b])
    m_VES = round(pd.DataFrame(data = m_VES, columns=['m_VES']), 2)
    
    #Добавим показатель степени для каждого измерения в таблице с данными метеостанции
    mm_VES = ()
    for j in range (1, 13):
        for i in range(len(Meteostation_XX.loc
                           [Meteostation_XX['Month'] == j]['V_VES'])):
            mm_VES = np.append([mm_VES], [m_VES.loc[j-1]])
    mm_VES = round(pd.DataFrame(data = mm_VES, columns=['mm_VES']), 2)
    Meteostation_XX = Meteostation_XX.join(mm_VES)
    
    #Пересчитай скорость ветра на высоту башни ВЭУ на площадке ВЭС
    V_VES_H = ()
    for i in range(len(Meteostation_XX)):
        V_VES_H = np.append([V_VES_H], [Meteostation_XX['V_VES'][i]*
                                        (float(Wind_Turbines[Select_WT][50])/10)
                                        **Meteostation_XX['mm_VES'][i]])
    V_VES_H = round(pd.DataFrame(data = V_VES_H, columns=['V_VES_H']), 2)
    
    #Рассчитай первичную выработку ВЭС
    WT_Power = ()
    xp = Wind_Turbines['V, m/s'][1:50]
    fp = Wind_Turbines[Select_WT][1:50]
    
    for i in range(len(V_VES_H)):
        if (float(V_VES_H['V_VES_H'][i])) < Wind_Turbines[Select_WT][51]:
            WT_Power = np.append([WT_Power], 0)
        else:
            WT_Power = np.append([WT_Power], 
                                 [np.interp(float(V_VES_H['V_VES_H'][i]), xp, fp)])
    WT_Power = round(pd.DataFrame(data = WT_Power, columns=['WT_Power']), 2)
    
    #Рассчитай финальную выработку ВЭС
    E_WT_prim = WT_Power.sum()*8760/len(WT_Power)
    Betta = 0.885
    E_WT_fin = Betta * E_WT_prim
    return(E_WT_fin)

In [ ]:
#Расчет выработки каждой ВЭУ из БД
All_WT_Energy = ()

All_WT = Wind_Turbines.columns.drop('V, m/s')
for i in range(len(All_WT)):
    Select_WT = All_WT[i]
    All_WT_Energy = (pd.DataFrame(data = 
                                  (np.append([All_WT_Energy], 
                                             round(Wind_calc(Select_WT, X, MX), 2))), 
                                  columns=['All_WT_Energy']))
All_WT_Energy = All_WT_Energy.join(pd.DataFrame(data = All_WT, columns = ['All_WT']))
All_WT_Energy = All_WT_Energy.join(pd.DataFrame(data = WT_Models_Power, columns = ['WT_Models_Power']))
All_WT_Energy = All_WT_Energy[['All_WT', 'WT_Models_Power', 'All_WT_Energy']]
#Расчет может занять время, приблизительно 1 секунда на 1 ВЭУ при кол-ве значений для 1 метеостанции - 2900 
#и производительности - уровня ноутбука Lenovo Legion 5 15ARH05 
#с процессором AMD Ryzen 5 4600H 

#Выбор ВЭУ в соответсвии с уровнем потребления
Final_Tab = ()
for i in range(len(All_WT_Energy)):
    if All_WT_Energy['All_WT_Energy'][i] > Cons:
        All_WT_Energy.drop(index = [i], inplace = True)
Final_Tab = pd.DataFrame(data = (list(All_WT_Energy['All_WT']).copy()), 
                         columns = ['All_WT'])
Final_Tab = Final_Tab.join(pd.DataFrame(data = list(All_WT_Energy['WT_Models_Power']).copy(), 
                                        columns = ['WT_Power']))
Final_Tab = Final_Tab.join(pd.DataFrame(data = list(All_WT_Energy['All_WT_Energy']).copy(), 
                                        columns = ['All_WT_Energy']))

#Рассчет необходимой выработки СЭС
All_Solar_Energy = ()
for i in range(len(Final_Tab)):
    All_Solar_Energy = np.append([All_Solar_Energy], 
                                 [round((Cons - Final_Tab['All_WT_Energy'][i]), 2)])
Final_Tab = Final_Tab.join(pd.DataFrame(data = All_Solar_Energy, 
                                        columns = ['All_Solar_Energy']))

#Функция по определению выработки СЭС
def PV_Energy(Peak_Power, X1, X2):
    X = (X1, X2)
    Optim_an = False
    Optim_az = False
    u=()
    Optim_angles = str(input('Оптимизировать угол установки? (Да/Нет)\n'))
    if Optim_angles.lower() == "да":
        Optim_an = True
    else:
        Angle = float(input('Задай угол установки:\n'))
    Optim_azimut = str(input('Оптимизировать азимут установки? (Да/Нет)\n'))
    if Optim_azimut.lower() == "да":
        Optim_az = True
    else:
        Azimut = float(input('Задай азимут установки:\n'))
    if (Optim_an is True) and Optim_az is True: #Оптим. все
        u = ('https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat='
        + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power)
        + '&pvtechchoice=crystSi&loss=14&optimalangles=1&outputformat=json')
    elif (Optim_an is False) and Optim_az is False: #Задай все
        u = ('https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat='
        + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power)
        + '&pvtechchoice=crystSi&loss=14&angle=' + str(Angle) + '&aspect='
        + str(Azimut) + '&outputformat=json')
    elif (Optim_an is True) and Optim_az is False: #Задай все
        u = ('https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat='
        + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power)
        + '&pvtechchoice=crystSi&loss=14&optimalinclination=1&aspect='
        + str(Azimut) + '&outputformat=json')
    elif (Optim_an is False) and Optim_az is True: #Задай все
        u = ('https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat='
        + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power)
        + '&pvtechchoice=crystSi&loss=14&angle=' + str(Angle) + '&aspect=0&outputformat=json')
    url = u
    r = requests.get(url)
    json = r.json()
    Z = json['outputs']['totals']['fixed']['E_y']
    Ugol = json['inputs']['mounting_system']['fixed']['slope']['value']
    Azimut = json['inputs']['mounting_system']['fixed']['azimuth']['value']
    return(Z, Ugol, Azimut)

#Функция по определению мощности СЭС из условия необходимой выработки
One_kW_PV = PV_Energy(1, X1, X2)
One_kW_PV_Energy = One_kW_PV[0]
PV_Ugol = One_kW_PV[1]
PV_Azimuth = One_kW_PV[2]
def PV_Power(Cons):
    Solar_Power = round(Cons/One_kW_PV_Energy, 2)
    return(Solar_Power)
print('Угол установки =', PV_Ugol, 'град.\n'
     'Азимут установки =', PV_Azimuth, ' град.')

#Рассчет необходимой мощности СЭС, формирование итоговой таблицы
All_Solar_Power = ()
for i in range(len(Final_Tab)):
    All_Solar_Power = np.append([All_Solar_Power], 
                                PV_Power(Final_Tab['All_Solar_Energy'][i]))
Final_Tab = Final_Tab.join(pd.DataFrame(data = All_Solar_Power, 
                                        columns = ['All_Solar_Power']))
for i in range(len(Final_Tab)):
    Final_Tab.loc[(Final_Tab.All_Solar_Power < 1), 'All_Solar_Power'] = 0 
    Final_Tab.loc[(Final_Tab.All_Solar_Energy < One_kW_PV_Energy), 
                  'All_Solar_Energy'] = 0 
Final_Tab = Final_Tab[['All_WT', 'WT_Power', 'All_Solar_Power', 
                       'All_WT_Energy', 'All_Solar_Energy']]
Final_Tab = Final_Tab.rename(columns = {'All_WT' : 'Модель ВЭУ', 
                                        'WT_Power' : 'ВЭУ, кВт', 
                                        'All_Solar_Power' : 'СЭС, кВт', 
                                        'All_WT_Energy' : 'Выработка ВЭС за 1 год, кВт*ч', 
                                        'All_Solar_Energy' : 'Выработка СЭС за 1 год, кВт*ч'})

#Добавлена СЭС, способная покрыть все потребление
Final_Tab = Final_Tab.append({'ВЭУ, кВт': '0', 'СЭС, кВт':PV_Power(Cons), 
                              'Выработка ВЭС за 1 год, кВт*ч': '0', 
                              'Выработка СЭС за 1 год, кВт*ч': Cons }, 
                             ignore_index=True)
Final_Tab

In [ ]:
Lifetime = int(input('Задай срок службы ГЭК, лет: '))
Rate = float(input('Задай ставку дисконтирования, %: '))
PR_25 = float(input('Задай КПД Солнечного Модуля после 25 лет службы, %: '))
OPEX_SES_1 = float(input('Задай операционные затраты на СЭС, Евро/кВт/год (рекомендуется - 5): '))
OPEX_VES_1 = float(input('Задай операционные затраты на ВЭС, Евро/кВт/год (рекомендуется - 30): '))

In [ ]:
#Функция расчета выработки СЭС за период эксплуатации
def Solar_Economy(SS_Energy_1, Lifetime=20, PR_25=80):
    SS_Lifetime = Lifetime
    degradation = ((100-PR_25)/25)
    Dgd = ()
    
    for i in range(SS_Lifetime):
        Dgd = np.append([Dgd], 100 - i * degradation)
    Dgd
    Solar_Economy_Table = pd.DataFrame(data = Dgd, columns = ['Dgd'])
    
    SS_Energy_PE = ()
    
    for i in range(SS_Lifetime):
        SS_Energy_PE = np.append([SS_Energy_PE], (SS_Energy_1 * Solar_Economy_Table['Dgd'][i]/100))
    Solar_Economy_Table = Solar_Economy_Table.join(pd.DataFrame(data = SS_Energy_PE, columns = ['SS_Energy_PE']))
    SS_Energy = round(sum(Solar_Economy_Table['SS_Energy_PE']), 1)
    return(SS_Energy, Solar_Economy_Table['SS_Energy_PE'])

#Рассчет выработки СЭС за период эксплуатации
SS_En = ()
for i in range(len(Final_Tab)):
    SS_En = np.append([SS_En], (Solar_Economy(Final_Tab['Выработка СЭС за 1 год, кВт*ч'][i], 
                                 Lifetime, PR_25)[0]))
Final_Tab = Final_Tab.join(pd.DataFrame(data = SS_En.copy(), 
                                        columns = ['Выработка СЭС за ПЭ, кВт*ч']))

#Рассчет выработки ВЭС за период эксплуатации
VES_En = ()
for i in range(len(Final_Tab)):
    VES_En = np.append([VES_En], 
                       Final_Tab['Выработка ВЭС за 1 год, кВт*ч'][i] * Lifetime)
Final_Tab = Final_Tab.join(pd.DataFrame(data = VES_En.copy(), columns = ['Выработка ВЭС за ПЭ, кВт*ч']))
for i in range(len(Final_Tab)-1):
    Final_Tab['Выработка ВЭС за ПЭ, кВт*ч'][i] = round(float(Final_Tab['Выработка ВЭС за ПЭ, кВт*ч'][i]), 2)
    
#Рассчет выработки ГЭК за период эксплуатации
GEK_En = ()
for i in range(len(Final_Tab)-1):
    GEK_En = np.append([GEK_En], (Final_Tab['Выработка ВЭС за ПЭ, кВт*ч'][i] 
                                  + Final_Tab['Выработка СЭС за ПЭ, кВт*ч'][i]))
Final_Tab = Final_Tab.join(pd.DataFrame(data = GEK_En, columns = ['Выработка ГЭК за ПЭ, кВт*ч']))
Final_Tab['Выработка ГЭК за ПЭ, кВт*ч'].iloc[-1] = Final_Tab['Выработка СЭС за ПЭ, кВт*ч'].iloc[-1]
Final_Tab['Выработка ВЭС за ПЭ, кВт*ч'].iloc[-1] = 0
#Одиночная СЭС - последняя в списке. Вместо сложения выработки СЭС + ВЭС, пришлось присвоить ячейке значение только СЭС
Final_Tab

In [ ]:
#Формируем таблицу для Экономического расчета всех вариантов ГЭК
Economy_Table = pd.DataFrame(data = Final_Tab['Модель ВЭУ'].copy(),
                             columns = ['Модель ВЭУ'])
Economy_Table = Economy_Table.join(pd.DataFrame(data = Final_Tab['ВЭУ, кВт'].copy(), 
                                                columns = ['ВЭУ, кВт']))
Economy_Table = Economy_Table.join(pd.DataFrame(data = Final_Tab['СЭС, кВт'].copy(), 
                                                columns = ['СЭС, кВт']))
Economy_Table = Economy_Table.join(pd.DataFrame(data = Final_Tab['Выработка ГЭК за ПЭ, кВт*ч'].copy(), 
                                                columns = ['Выработка ГЭК за ПЭ, кВт*ч']))

#Импортирует цены на выбранные модели ВЭУ из списка цен WT_Prices.xlsx
WT_Prices = pd.read_excel('WT_Prices.xlsx')
Sel_WT = ()
for i in range(len(Economy_Table)):
    for j in range(len(WT_Prices)):
        if Economy_Table['Модель ВЭУ'][i] == WT_Prices['Модель ВЭУ'][j]:
            Sel_WT = np.append([Sel_WT], (WT_Prices['Цена, €'][j]))
Economy_Table = Economy_Table.join((pd.DataFrame(data = Sel_WT, columns = ['Стоимость ВЭС, €'])))
Economy_Table['Стоимость ВЭС, €'].iloc[-1] = 0

#Определяет стоимость СЭС на основании ряда цен из файла Solar_Prices.xlsx
Solar_Prices = pd.read_excel('Solar_Prices.xlsx')
Sel_SS = ()

for i in range(len(Economy_Table)):
    if Economy_Table['СЭС, кВт'][i] != 0:
        Sel_SS = np.append([Sel_SS], Economy_Table['СЭС, кВт'][i]*
                           (np.interp(float(Economy_Table['СЭС, кВт'][i]), 
                                      Solar_Prices['Мощность, кВт'], Solar_Prices['Цена, €'])))
    else:
        Sel_SS = np.append([Sel_SS], 0)
Economy_Table = Economy_Table.join((pd.DataFrame(data = Sel_SS, columns = ['Стоимость СЭС, €'])))
for i in range(len(Economy_Table)):
    Economy_Table['Стоимость СЭС, €'][i] = round(Economy_Table['Стоимость СЭС, €'][i], 2) 

#Определяет стоимость ГЭК
GEK_Costs = ()
for i in range(len(Economy_Table)):
    GEK_Costs = np.append([GEK_Costs], Economy_Table['Стоимость ВЭС, €'][i] 
                          + Economy_Table['Стоимость СЭС, €'][i])
Economy_Table = Economy_Table.join((pd.DataFrame(data = GEK_Costs, columns = ['Стоимость ГЭК, €'])))
Economy_Table

In [ ]:
#Функция по расчету LCOE всего ГЭК (СЭС или ВЭС могу принимать 0 значения)
def LCOE(Lifetime=20, Invest=0, Power_SES=0, Energy_SES_1=0, Power_VES=0, 
         Energy_VES_1=0, Rate=10, PR_25=80, OPEX_SES_1=5, OPEX_VES_1=30):
    Year = range(Lifetime)
    LCOE_df = pd.DataFrame(data = Year, columns = ['Year'])
    Invests = [0] * int(Lifetime+1)
    Invests[0] = Invest
    LCOE_df = LCOE_df.join(pd.DataFrame(data = Invests, columns = ['Invests']))
    Disc_rate = ()
    for i in range(len(LCOE_df)):
        Disc_rate = np.append([Disc_rate], Rate/100*LCOE_df['Year'][i])
    LCOE_df = LCOE_df.join(pd.DataFrame(data = Disc_rate, columns = ['Disc_rate']))
    OPEX_SES_11 = OPEX_SES_1*float(Power_SES)
    OPEX_SES = [OPEX_SES_11] * int(Lifetime+1)
    LCOE_df = LCOE_df.join(pd.DataFrame(data = OPEX_SES, columns = ['OPEX_SES']))
    OPEX_VES_11 = OPEX_VES_1*float(Power_VES)
    OPEX_VES = [OPEX_VES_11] * int(Lifetime+1)
    LCOE_df = LCOE_df.join(pd.DataFrame(data = OPEX_VES, columns = ['OPEX_VES']))    
    Energy_SES = ()
    for i in range(len(LCOE_df)):
        Energy_SES = np.append([Energy_SES], Energy_SES_1*
                               (100-LCOE_df['Year'][i]*(100-PR_25)/25)/100)
    LCOE_df = LCOE_df.join(pd.DataFrame(data = Energy_SES, columns = ['Energy_SES']))
    Energy_VES = [Energy_VES_1] * int(Lifetime+1)
    LCOE_df = LCOE_df.join(pd.DataFrame(data = Energy_VES, columns = ['Energy_VES']))    
    Energy_GEK = ()
    for i in range(len(LCOE_df)):
        Energy_GEK = np.append([Energy_GEK], float(LCOE_df['Energy_VES'][i])+float(LCOE_df['Energy_SES'][i]))
    LCOE_df = LCOE_df.join(pd.DataFrame(data = Energy_GEK, columns = ['Energy_GEK']))    
    Cislit = ()
    for i in range(len(LCOE_df)):
        Cislit = np.append([Cislit], (LCOE_df['Invests'][i]+LCOE_df['OPEX_SES'][i]+
                                      LCOE_df['OPEX_VES'][i])/(1+LCOE_df['Disc_rate'][i])**LCOE_df['Year'][i])
    LCOE_df = LCOE_df.join(pd.DataFrame(data = Cislit, columns = ['Cislit']))    
    Znamen = ()
    for i in range(len(LCOE_df)):
        Znamen = np.append([Znamen], LCOE_df['Energy_GEK'][i]/((1+LCOE_df['Disc_rate'][i])**LCOE_df['Year'][i]))
    LCOE_df = LCOE_df.join(pd.DataFrame(data = Znamen, columns = ['Znamen']))    
    LCOE = sum(LCOE_df['Cislit'])/sum(LCOE_df['Znamen']) 
    return(LCOE)

#Расчет LCOE вариантов ГЭК
GEK_LCOE = ()
for i in range(len(Economy_Table)):
    GEK_LCOE = np.append([GEK_LCOE], LCOE(Lifetime, Economy_Table['Стоимость ГЭК, €'][i], 
                                          Economy_Table['СЭС, кВт'][i], Final_Tab['Выработка СЭС за 1 год, кВт*ч'][i], 
                                          Economy_Table['ВЭУ, кВт'][i], Final_Tab['Выработка ВЭС за 1 год, кВт*ч'][i], 
                                          Rate, PR_25, OPEX_SES_1, OPEX_VES_1))
Economy_Table = Economy_Table.join(pd.DataFrame(data = GEK_LCOE, columns = ['LCOE, €/кВт*ч']))
Economy_Table = Economy_Table.sort_values(by='LCOE, €/кВт*ч')
Economy_Table.index = np.arange(len(Economy_Table))
Economy_Table

In [ ]:
#Строит график LCOE для всех ГЭК
names = ()
for i in range(len(Economy_Table)):
    names = np.append([names], Economy_Table['Модель ВЭУ'][i])
#Подписывает одиночную СЭС
for i in range(len(names)):
    if names[i] == 'nan':
        names[i] = ('СЭС ' + str(Economy_Table['СЭС, кВт'][i]))
    else:
        names[i] = (names[i] + ' + СЭС ' + str(Economy_Table['СЭС, кВт'][i]))
values = ()
for i in range(len(Economy_Table)):
    values = np.append([values], Economy_Table['LCOE, €/кВт*ч'][i])
values
fig = plt.figure(dpi=120)
fig = plt.scatter(names, values)
plt.xticks(rotation=90)
plt.grid(True)